<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/FGSM_attack_and_PGD_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers
from absl import app, flags

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
cnn_model = keras.models.load_model('Desktop/cnn_model_cifar10.h5')
vgg_16 = keras.models.load_model('Desktop/vgg16_cifar10.h5')
resnet = keras.models.load_model('Desktop/resnet_cifar10.h5')

In [14]:
cifar_10 = tf.keras.datasets.cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar_10
def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  return X_train,y_train,X_test,y_test
X_train,y_train,X_test,y_test = pre_processing_cifar10(X_train,y_train,X_test,y_test,3)

In [15]:

def make_predictions(model , image , true_value):

  labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  predicted_label = labels[predicted_label_index]
  true_label = labels[true_label_index]
  print()
  print()
  print('Predicted Label -',predicted_label,' Probability -',probability)
  print('True Label -',true_label)
  print()
  print()

  return predicted_label , probability 


In [16]:
result = {} # key  - (model , epsilon ,fgsm_misclassification , pdg_missclassification)

#FGSM and PGD attacks on standard cnn model

In [17]:
from keras.preprocessing.image import img_to_array

epsilon_end = 0.8
logits_cnn_model = tf.keras.Model(cnn_model.input,cnn_model.layers[-1].output)
total_images = 200


print('Standard CNN result')
epsilon = 0.1
while(epsilon <= epsilon_end):

  fgsm_counter = 0
  pgd_counter = 0
  print("Epsilon value - ",epsilon)
  print()
  for image_index in range(total_images):
  
    print('-------------------------------')
    print('Image index :',image_index)
    print('Original Sample')
    image  = X_test[image_index]
    image = img_to_array(image)
    image = image.reshape(1, 32, 32, 3)
    true_value = y_test[image_index]
    original_prediction = make_predictions(cnn_model,image,true_value)
    fgsm_sample = fast_gradient_method(logits_cnn_model, image, epsilon, np.inf, targeted=False)
    pgd_sample = projected_gradient_descent(cnn_model, image, epsilon, 0.01, 40, np.inf)
    print()
    print('Untargeted FGSM Adverserial Sample')
    fgsm_prediction = make_predictions(cnn_model , fgsm_sample , true_value)
    print()
    print('PGD Adverserial Sample')
    pgd_prediction = make_predictions(cnn_model , pgd_sample , true_value)
    print()
  
    if(pgd_prediction[0]!=original_prediction[0]):
     pgd_counter+=1
    if(fgsm_prediction[0]!=original_prediction[0]):
      fgsm_counter+=1
  result[('cnn',epsilon)] = (fgsm_counter , pgd_counter)
  epsilon+=0.1

print(result)

Streaming output truncated to the last 5000 lines.


Predicted Label - frog  Probability - 0.77545637
True Label - frog



-------------------------------
Image index : 8
Original Sample


Predicted Label - cat  Probability - 0.64392453
True Label - cat



Untargeted FGSM Adverserial Sample


Predicted Label - deer  Probability - 0.73903334
True Label - cat



PGD Adverserial Sample


Predicted Label - deer  Probability - 0.55577177
True Label - cat



-------------------------------
Image index : 9
Original Sample


Predicted Label - automobile  Probability - 0.9783589
True Label - automobile



Untargeted FGSM Adverserial Sample


Predicted Label - automobile  Probability - 0.6942384
True Label - automobile



PGD Adverserial Sample


Predicted Label - automobile  Probability - 0.89535385
True Label - automobile



-------------------------------
Image index : 10
Original Sample


Predicted Label - airplane  Probability - 0.60739666
True Label - airplane



Untargeted FGSM Adverseria

In [18]:
result

{('cnn', 0.1): (7, 8),
 ('cnn', 0.2): (14, 15),
 ('cnn', 0.30000000000000004): (19, 19),
 ('cnn', 0.4): (23, 23),
 ('cnn', 0.5): (25, 23),
 ('cnn', 0.6): (33, 23),
 ('cnn', 0.7): (35, 23),
 ('cnn', 0.7999999999999999): (42, 23)}

{('cnn', 0.1): (23, 24),
 ('cnn', 0.2): (37, 39),
 ('cnn', 0.30000000000000004): (45, 46),
 ('cnn', 0.4): (52, 53),
 ('cnn', 0.5): (58, 53),
 ('cnn', 0.6): (69, 53),
 ('cnn', 0.7): (75, 53),
 ('cnn', 0.7999999999999999): (85, 53)}

#FGSM and PGD attacks on vgg16

In [8]:
logits_vgg_16 = tf.keras.Model(vgg_16.input,vgg_16.layers[-1].output)
print('VGG16 result')
print()
epsilon = 0.1
while(epsilon <= epsilon_end):
  fgsm_counter = 0
  pgd_counter = 0
  print("Epsilon - ",epsilon)
  for image_index in range(total_images):
  
    print('-------------------------------')
    print()
    print('Image index :',image_index)
    print('Original Sample')
    image  = X_test[image_index]
    image = img_to_array(image)
    image = image.reshape(1, 32, 32, 3)
    true_value = y_test[image_index]
    original_prediction = make_predictions(vgg_16,image,true_value)
    fgsm_sample = fast_gradient_method(logits_vgg_16, image, epsilon, np.inf, targeted=False)
    pgd_sample = projected_gradient_descent(vgg_16, image, epsilon, 0.01, 40, np.inf)
    print()
    print()
    print('Untargeted FGSM Adverserial Sample')
    fgsm_prediction = make_predictions(vgg_16 , fgsm_sample , true_value)
    print()
    print('PGD Adverserial Sample')
    pgd_prediction = make_predictions(vgg_16 , pgd_sample , true_value)
    print()
  
    if(pgd_prediction[0]!=original_prediction[0]):
      pgd_counter+=1

    if(fgsm_prediction[0]!=original_prediction[0]):
      fgsm_counter+=1
     
  result[('vgg16',epsilon)] = (fgsm_counter , pgd_counter)
  epsilon+=0.1

Streaming output truncated to the last 5000 lines.
Predicted Label - bird  Probability - 0.97796714
True Label - bird



-------------------------------

Image index : 130
Original Sample


Predicted Label - deer  Probability - 0.99929893
True Label - deer




Untargeted FGSM Adverserial Sample


Predicted Label - deer  Probability - 0.99702483
True Label - deer



PGD Adverserial Sample


Predicted Label - deer  Probability - 0.9968966
True Label - deer



-------------------------------

Image index : 131
Original Sample


Predicted Label - automobile  Probability - 0.9999087
True Label - automobile




Untargeted FGSM Adverserial Sample


Predicted Label - automobile  Probability - 0.9998714
True Label - automobile



PGD Adverserial Sample


Predicted Label - automobile  Probability - 0.9998697
True Label - automobile



-------------------------------

Image index : 132
Original Sample


Predicted Label - ship  Probability - 0.9999944
True Label - ship




Untargeted FGSM Adverser

In [9]:
result

{('cnn', 0.1): (7, 8),
 ('cnn', 0.2): (14, 15),
 ('cnn', 0.30000000000000004): (19, 19),
 ('cnn', 0.4): (23, 23),
 ('cnn', 0.5): (25, 23),
 ('cnn', 0.6): (33, 23),
 ('cnn', 0.7): (35, 23),
 ('cnn', 0.7999999999999999): (42, 23),
 ('vgg16', 0.1): (6, 6),
 ('vgg16', 0.2): (9, 11),
 ('vgg16', 0.30000000000000004): (14, 14),
 ('vgg16', 0.4): (15, 16),
 ('vgg16', 0.5): (21, 16),
 ('vgg16', 0.6): (23, 16),
 ('vgg16', 0.7): (26, 16),
 ('vgg16', 0.7999999999999999): (28, 16)}

#FGSM and PGD attacks on resnet model

In [10]:
logits_resnet = tf.keras.Model(resnet.input,resnet.layers[-1].output)

print('Resnet result')
print()
epsilon = 0.1

while(epsilon <= epsilon_end):
  print("Epsilon - ",epsilon)
  fgsm_counter = 0
  pgd_counter = 0
  for image_index in range(total_images):
  
    print('-------------------------------')
    print()
    print('Image index :',image_index)
    print()
    print('Original Sample')
    image  = X_test[image_index]
    image = img_to_array(image)
    image = image.reshape(1, 32, 32, 3)
    true_value = y_test[image_index]
    original_prediction = make_predictions(resnet,image,true_value)
    fgsm_sample = fast_gradient_method(logits_resnet, image, epsilon, np.inf, targeted=False)
    pgd_sample = projected_gradient_descent(resnet, image, epsilon , 0.01, 40, np.inf)
    print()
    print('Untargeted FGSM Adverserial Sample')
    fgsm_prediction = make_predictions(resnet , fgsm_sample , true_value)
    print()
    print('PGD Adverserial Sample')
    pgd_prediction = make_predictions(resnet , pgd_sample , true_value)
    print()
  
    if(pgd_prediction[0]!=original_prediction[0]):
      pgd_counter+=1

    if(fgsm_prediction[0]!=original_prediction[0]):
      fgsm_counter+=1
  result[('resnet',epsilon)] = (fgsm_counter , pgd_counter)
  epsilon+=0.1


Streaming output truncated to the last 5000 lines.


Predicted Label - bird  Probability - 0.9780086
True Label - airplane



PGD Adverserial Sample


Predicted Label - bird  Probability - 0.9884239
True Label - airplane



-------------------------------

Image index : 22

Original Sample


Predicted Label - deer  Probability - 0.9989918
True Label - deer



Untargeted FGSM Adverserial Sample


Predicted Label - airplane  Probability - 0.5524384
True Label - deer



PGD Adverserial Sample


Predicted Label - deer  Probability - 0.8897444
True Label - deer



-------------------------------

Image index : 23

Original Sample


Predicted Label - truck  Probability - 1.0
True Label - truck



Untargeted FGSM Adverserial Sample


Predicted Label - truck  Probability - 0.9999999
True Label - truck



PGD Adverserial Sample


Predicted Label - truck  Probability - 1.0
True Label - truck



-------------------------------

Image index : 24

Original Sample


Predicted Label - deer  Probabili

In [11]:
result

{('cnn', 0.1): (7, 8),
 ('cnn', 0.2): (14, 15),
 ('cnn', 0.30000000000000004): (19, 19),
 ('cnn', 0.4): (23, 23),
 ('cnn', 0.5): (25, 23),
 ('cnn', 0.6): (33, 23),
 ('cnn', 0.7): (35, 23),
 ('cnn', 0.7999999999999999): (42, 23),
 ('vgg16', 0.1): (6, 6),
 ('vgg16', 0.2): (9, 11),
 ('vgg16', 0.30000000000000004): (14, 14),
 ('vgg16', 0.4): (15, 16),
 ('vgg16', 0.5): (21, 16),
 ('vgg16', 0.6): (23, 16),
 ('vgg16', 0.7): (26, 16),
 ('vgg16', 0.7999999999999999): (28, 16),
 ('resnet', 0.1): (5, 5),
 ('resnet', 0.2): (8, 8),
 ('resnet', 0.30000000000000004): (12, 12),
 ('resnet', 0.4): (20, 22),
 ('resnet', 0.5): (26, 22),
 ('resnet', 0.6): (30, 22),
 ('resnet', 0.7): (36, 22),
 ('resnet', 0.7999999999999999): (38, 22)}